<a href="https://colab.research.google.com/github/Rashmi-03/Glowup-cosmetics-/blob/main/Glowup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import display, HTML, clear_output
from google.colab import output
import re # For basic regex validation

# --- Data ---
USERS = [{'email': 'user@example.com', 'password': 'password'}]
PRODUCTS = [
    {'id': 'lipstick1', 'name': "Velvet Matte Lipstick", 'desc': "Smooth & Bold", 'price': "₹499", 'manufacturer': "Mumbai, India", 'image': "https://th.bing.com/th/id/OIP.hzpzCmM5Z463iNfdrSeymgHaHa?w=196&h=196&c=7&r=0&o=5&pid=1.7"},
    {'id': 'lipstick2', 'name': "Glossy Shine Lipstick", 'desc': "Gloss finish", 'price': "₹599", 'manufacturer': "Delhi, India", 'image': "https://th.bing.com/th/id/OIP.LQSJvVBDhEFCPfNZtx8eEgAAAA?w=183&h=182&c=7&r=0&o=5&pid=1.7"},
    {'id': 'mascara1', 'name': "Waterproof Mascara", 'desc': "Long-lasting", 'price': "₹699", 'manufacturer': "Pune, India", 'image': "https://th.bing.com/th/id/OIP.dN1HGcOT1_cg3FW8wSv-RgHaHa?w=206&h=206&c=7&r=0&o=5&pid=1.7"},
    {'id': 'serum1', 'name': "Vitamin C Serum", 'desc': "Glowing skin", 'price': "₹799", 'manufacturer': "Goa, India", 'image': "https://th.bing.com/th/id/OIP.Fz7N8J7-_clVLdiKSjNb-AHaJ4?w=144&h=192&c=7&r=0&o=5&pid=1.7"},
    {'id': 'serum2', 'name': "Hyaluronic Acid Serum", 'desc': "Hydration Boost", 'price': "₹899", 'manufacturer': "Chennai, India", 'image': "https://th.bing.com/th/id/OIP.xrpwMHMld_vIjfO6q2a2xwHaHa?w=201&h=201&c=7&r=0&o=5&pid=1.7"},
    {'id': 'dryer1', 'name': "Hair Dryer Pro 2500W", 'desc': "Fast Drying", 'price': "₹1499", 'manufacturer': "Nagpur, India", 'image': "https://th.bing.com/th/id/OIP.u5uxyVzOufE53Z2-8YFFGAAAAA?w=165&h=180&c=7&r=0&o=5&pid=1.7"},
    {'id': 'dryer2', 'name': "Mini Travel Hair Dryer", 'desc': "Compact Power", 'price': "₹999", 'manufacturer': "Indore, India", 'image': "https://th.bing.com/th/id/OIP.iezYI-UA2DoUokf6zM_rswHaHa?w=216&h=216&c=7&r=0&o=5&pid=1.7"}
]

MOCK_ORDERS = [ # Mock orders data, in a real app this would come from a database
    {
        'order_id': 'ORDGLW001', 'date': '2025-05-15', 'status': 'Delivered',
        'items': [
            {'name': "Velvet Matte Lipstick", 'quantity': 1, 'price_each': "₹499"},
            {'name': "Waterproof Mascara", 'quantity': 1, 'price_each': "₹699"}
        ],
        'total_amount': "₹1198.00"
    },
    {
        'order_id': 'ORDGLW002', 'date': '2025-05-20', 'status': 'Shipped',
        'items': [
            {'name': "Vitamin C Serum", 'quantity': 2, 'price_each': "₹799"}
        ],
        'total_amount': "₹1598.00"
    },
    {
        'order_id': 'ORDGLW003', 'date': '2025-05-25', 'status': 'Processing',
        'items': [
            {'name': "Hair Dryer Pro 2500W", 'quantity': 1, 'price_each': "₹1499"},
            {'name': "Glossy Shine Lipstick", 'quantity': 1, 'price_each': "₹599"}
        ],
        'total_amount': "₹2098.00"
    }
]


# --- Session Simulation ---
session = {
    'logged_in': False, 'current_user_email': None,
    'current_view': 'login', 'selected_product': None, 'cart': [],
    'direct_buy_item_details': None
}

# --- HTML/CSS for the Frontend ---
BASE_CSS = """
<style>
    @import url('https://fonts.googleapis.com/css2?family=Open+Sans:wght@400;700&family=Playfair+Display:wght@700&display=swap');
    * { margin: 0; padding: 0; box-sizing: border-box; }
    body { font-family: 'Open Sans', sans-serif; background: linear-gradient(to right, #fff5f7, #ffe3ec); color: #333; padding: 1rem; }
    header { text-align: center; padding: 2rem; background-color: #fff0f6; margin-bottom: 0; border-radius: 8px 8px 0 0; box-shadow: 0 2px 5px rgba(0,0,0,0.05); }
    header h1 { font-family: 'Playfair Display', serif; color: #cc3366; font-size: 3rem; text-shadow: 1px 1px 2px rgba(0,0,0,0.1); }

    .main-nav {
        text-align: center;
        padding: 0.75rem 0;
        background-color: #fceef3; /* Lighter pink for nav */
        border-bottom: 1px solid #f7d9e5;
        margin-bottom: 1.5rem; /* Space below nav */
        border-radius: 0 0 8px 8px;
        box-shadow: 0 2px 5px rgba(0,0,0,0.05);
    }
    .main-nav a {
        margin: 0 0.75rem;
        color: #cc3366;
        text-decoration: none;
        font-weight: bold;
        padding: 0.5rem 0.75rem;
        border-radius: 4px;
        transition: background-color 0.3s ease, color 0.3s ease;
    }
    .main-nav a:hover, .main-nav a.active { /* Added .active for current page */
        background-color: #cc3366;
        color: white;
    }

    .form-container { max-width: 450px; margin: 2rem auto; background: white; padding: 2rem; border-radius: 10px; box-shadow: 0 4px 15px rgba(0,0,0,0.1); animation: fadeIn 0.5s ease-out; }
    .form-container h2 { text-align: center; color: #cc3366; margin-bottom: 1.5rem; font-family: 'Playfair Display', serif; }
    .form-container input { width: 100%; padding: 0.85rem; margin-bottom: 1rem; border: 1px solid #ddd; border-radius: 5px; font-size: 1rem; }
    .form-container button, .detail-view button.checkout-btn { width: 100%; padding: 0.85rem; background-color: #cc3366; color: white; border: none; border-radius: 5px; cursor: pointer; font-size: 1.1rem; transition: background-color 0.3s ease; margin-top: 0.5rem; }
    .form-container button:hover, .detail-view button.checkout-btn:hover { background-color: #a0204d; }
    .form-container .secondary-action-btn { background-color: #777; margin-top: 0.75rem; }
    .form-container .secondary-action-btn:hover { background-color: #555; }
    .payment-total, .payment-item-summary { text-align: center; font-size: 1.2rem; font-weight: bold; color: #333; margin-bottom: 0.5rem; padding: 0.75rem 0; }
    .payment-total { border-top: 1px solid #eee; border-bottom: 1px solid #eee; margin-bottom: 1.5rem;}
    .payment-item-summary { font-size: 1rem; font-weight: normal; color: #555; margin-bottom:1.5rem; }

    .form-toggle { text-align: center; margin-top: 1rem; font-size: 0.9rem; color: #666; }
    .form-toggle a { color: #cc3366; text-decoration: none; }
    .search-bar { text-align: center; margin: 2rem 0; }
    .search-bar input { width: 60%; max-width: 500px; padding: 0.75rem 1rem; font-size: 1rem; border-radius: 25px; border: 1px solid #ccc; box-shadow: 0 2px 5px rgba(0,0,0,0.05); }
    .container { display: flex; flex-wrap: wrap; justify-content: center; gap: 2rem; padding: 1rem; animation: fadeIn 0.5s ease-out; }
    .product { background: white; border-radius: 10px; box-shadow: 0 4px 15px rgba(0,0,0,0.1); width: 250px; overflow: hidden; transition: transform 0.3s ease, box-shadow 0.3s ease; cursor: pointer; display: flex; flex-direction: column; }
    .product:hover { transform: translateY(-5px); box-shadow: 0 6px 20px rgba(0,0,0,0.15); }
    .product img { width: 100%; height: 200px; object-fit: cover; border-bottom: 1px solid #eee; }
    .product-info { padding: 1rem; text-align: center; flex-grow: 1; display: flex; flex-direction: column; justify-content: space-between; }
    .product-info h3 { color: #cc3366; margin-bottom: 0.5rem; font-family: 'Playfair Display', serif; }
    .product-info p { font-size: 0.9rem; color: #666; margin-bottom: 0.75rem; }
    .price { font-weight: bold; color: #333; font-size: 1.1rem; }
    .detail-view { background: white; margin: 1rem auto; padding: 2.5rem; border-radius: 10px; max-width: 700px; box-shadow: 0 4px 20px rgba(0,0,0,0.1); display: flex; flex-direction: column; align-items: center; text-align: center; animation: fadeIn 0.5s ease-out; }
    .detail-view img { width: 80%; max-width: 400px; border-radius: 10px; margin-bottom: 1.5rem; box-shadow: 0 2px 8px rgba(0,0,0,0.1); }
    .detail-view h2 { color: #cc3366; margin-bottom: 0.75rem; font-family: 'Playfair Display', serif; font-size: 2rem; }
    .detail-view p { font-size: 1rem; color: #555; margin-bottom: 0.5rem; }
    .detail-view .price { font-size: 1.4rem; margin-top: 1rem; margin-bottom: 1.5rem; color: #cc3366; }
    .btn { margin-top: 1rem; padding: 0.85rem 1.8rem; background-color: #cc3366; color: white; border: none; border-radius: 5px; cursor: pointer; font-size: 1rem; transition: background-color 0.3s ease, transform 0.2s ease; }
    .btn:hover { background-color: #a0204d; transform: translateY(-2px); }
    .back-btn { background: #555; }
    .back-btn:hover { background: #333; }
    .detail-buttons-container { display: flex; gap: 1rem; margin-top: 1rem; margin-bottom: 1rem; justify-content: center; flex-wrap: wrap; }
    .detail-buttons-container .btn { margin-top: 0; }
    .buy-now-btn { background-color: #28a745; }
    .buy-now-btn:hover { background-color: #218838; }
    .cart-summary { text-align: center; margin-top: 2rem; padding-top: 1rem; border-top: 1px solid #eee; }
    .cart-summary p { font-size: 1.3rem; color: #cc3366; font-weight: bold; }
    .cart-items { list-style: none; padding: 0; margin-top: 1.5rem; width: 100%; }
    .cart-items li { background: #fdf5f7; margin-bottom: 0.75rem; padding: 1rem 1.25rem; border-radius: 5px; display: flex; justify-content: space-between; align-items: center; box-shadow: 0 1px 3px rgba(0,0,0,0.05); font-size: 1rem; flex-wrap: wrap; }
    .cart-items .remove-btn { background-color: #dc3545; padding: 0.4rem 0.8rem; font-size: 0.85rem; margin-top: 0; }
    .cart-items .remove-btn:hover { background-color: #c82333; }
    .error-message { color: red; text-align: center; margin-bottom: 1rem; font-weight: bold; }
    .success-message { color: green; text-align: center; margin-bottom: 1rem; font-weight: bold; }
    .confirmation-view { text-align: center; padding: 2rem; } /* Applied to .detail-view for consistency */
    .confirmation-view h2 { color: #28a745; font-family: 'Playfair Display', serif; margin-bottom: 1rem;}
    .confirmation-view p { margin-bottom: 1rem; font-size: 1.1rem;}

    .orders-container { max-width: 800px; margin: 1rem auto; background: white; padding: 2rem; border-radius: 10px; box-shadow: 0 4px 15px rgba(0,0,0,0.1); animation: fadeIn 0.5s ease-out; }
    .orders-container h2 { text-align: center; color: #cc3366; margin-bottom: 1.5rem; font-family: 'Playfair Display', serif; }
    .order-card { border: 1px solid #f0d9e3; border-radius: 8px; padding: 1.5rem; margin-bottom: 1.5rem; background-color: #fffafd; box-shadow: 0 2px 8px rgba(0,0,0,0.07); }
    .order-card h3 { color: #a0204d; margin-bottom: 0.75rem; font-size: 1.3rem;}
    .order-card h4 { margin-top: 1rem; margin-bottom: 0.5rem; color: #555; font-size: 1rem; border-top: 1px dashed #eee; padding-top: 0.75rem;}
    .order-card ul { list-style: none; padding-left: 0; margin-bottom: 0.75rem;}
    .order-card ul li { padding: 0.3rem 0; font-size: 0.9rem; color: #444; display: flex; justify-content: space-between;}
    .order-card p { margin-bottom: 0.35rem; font-size: 0.95rem; }
    .order-card .order-total { font-weight: bold; color: #333; margin-top:1rem; font-size:1.1rem; text-align:right; border-top: 1px solid #f0d9e3; padding-top: 0.75rem;}
    .order-card .order-status { font-weight: bold; color: #28a745; } /* Green for delivered/shipped */
    .order-card .order-status.processing { color: #ffc107; } /* Yellow for processing */
    .no-orders { text-align: center; font-size: 1.1rem; color: #666; margin-top: 2rem; padding:2rem; background-color:#fffafd; border-radius:8px;}

    @keyframes fadeIn { from { opacity: 0; transform: translateY(10px); } to { opacity: 1; transform: translateY(0); } }
</style>
"""

# --- Helper Functions to Generate HTML Views ---

def get_login_form_html(error_message="", success_message=""):
    # CORRECTED: Removed extra single quote in button's onclick
    return f"""<div id="loginForm" class="form-container"><h2>Login</h2>{f'<p class="error-message">{error_message}</p>' if error_message else ""}{f'<p class="success-message">{success_message}</p>' if success_message else ""}<input type="text" placeholder="Email" id="login_email"><input type="password" placeholder="Password" id="login_password"><button onclick="google.colab.kernel.invokeFunction('login_submit', [document.getElementById('login_email').value, document.getElementById('login_password').value], {{}});\">Login</button><div class="form-toggle"><p>Don't have an account? <a href="#" onclick="google.colab.kernel.invokeFunction('show_signup_view', [], {{}}); return false;">Sign Up</a></p></div></div>"""

def get_signup_form_html(error_message="", success_message=""):
    # CORRECTED: Removed extra single quote in button's onclick
    return f"""<div id="signupForm" class="form-container"><h2>Sign Up</h2>{f'<p class="error-message">{error_message}</p>' if error_message else ""}{f'<p class="success-message">{success_message}</p>' if success_message else ""}<input type="text" placeholder="Email" id="signup_email"><input type="password" placeholder="Password" id="signup_password"><input type="password" placeholder="Confirm Password" id="signup_confirm_password"><button onclick="google.colab.kernel.invokeFunction('signup_submit', [document.getElementById('signup_email').value, document.getElementById('signup_password').value, document.getElementById('signup_confirm_password').value], {{}});\">Sign Up</button><div class="form-toggle"><p>Already have an account? <a href="#" onclick="google.colab.kernel.invokeFunction('show_login_view', [], {{}}); return false;">Login</a></p></div></div>"""

def get_html_template(main_content, current_view_name, error_message="", success_message=""):
    header_html = "<h1>GlowUp Cosmetics</h1>"
    auth_section_html = ""
    store_section_display = "none" # Default for main content area
    nav_html = ""

    if session.get('logged_in'):
        current_view = session.get('current_view', '')
        nav_links = [
            f"<a href='#' class='{'active' if current_view == 'shop' else ''}' onclick=\"google.colab.kernel.invokeFunction('show_shop_view', [], {{}}); return false;\">Shop</a>",
            f"<a href='#' class='{'active' if current_view == 'cart' else ''}' onclick=\"google.colab.kernel.invokeFunction('show_cart', [], {{}}); return false;\">Cart ({len(session.get('cart', []))})</a>",
            f"<a href='#' class='{'active' if current_view == 'orders' else ''}' onclick=\"google.colab.kernel.invokeFunction('show_orders_view', [], {{}}); return false;\">Your Orders</a>"
        ]
        if session.get('current_user_email'):
            nav_links.append(f"<a href='#' onclick=\"google.colab.kernel.invokeFunction('logout_user', [], {{}}); return false;\">Logout ({session['current_user_email'].split('@')[0]})</a>") # Show only username part
        nav_html = f"<nav class='main-nav'>{' '.join(nav_links)}</nav>"

    if current_view_name == 'login': auth_section_html = get_login_form_html(error_message=error_message, success_message=success_message)
    elif current_view_name == 'signup': auth_section_html = get_signup_form_html(error_message=error_message, success_message=success_message)
    elif current_view_name in ['shop', 'detail', 'cart', 'payment', 'confirmation', 'orders']: store_section_display = "block"

    return f"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8" /><meta name="viewport" content="width=device-width, initial-scale=1.0"/><title>GlowUp Cosmetics</title>{BASE_CSS}</head><body><header>{header_html}</header>{nav_html}<div id="storeSection" style="display: {store_section_display};">{main_content if current_view_name not in ['login', 'signup'] else ""}</div>{auth_section_html if current_view_name in ['login', 'signup'] else ""}</body></html>"""

def get_shop_html(products_to_display, search_query=""):
    # CORRECTED: Removed extra single quote in input's oninput
    product_cards_html = "".join([f"""<div class="product" onclick="google.colab.kernel.invokeFunction('show_product_detail', ['{p['id']}'], {{}});"> <img src="{p['image']}" alt="{p['name']}" /> <div class="product-info"> <h3>{p['name']}</h3> <p>{p['desc']}</p> <p class="price">{p['price']}</p> </div></div>""" for p in products_to_display]) if products_to_display else "<p style='text-align: center; margin-top: 2rem; color: #cc3366;'>No products found.</p>"
    return f"""<div class="search-bar"><input type="text" id="searchInput" placeholder="Search for Lipstick, Serum, Hair Dryer..." value="{search_query}" oninput="google.colab.kernel.invokeFunction('filter_products', [this.value], {{}});\"></div><section class="container" id="productContainer">{product_cards_html}</section>"""

def get_detail_html(product):
    if not product: return "<p style='text-align: center; color: #cc3366;'>Product not found. <button class='btn back-btn' onclick=\"google.colab.kernel.invokeFunction('show_shop_view', [], {{}})\">Back to Shop</button></p>"
    return f"""<section class="detail-view" id="detailView"><img src="{product['image']}" alt="{product['name']}"><h2>{product['name']}</h2><p>{product['desc']}</p><p class="price">Price: {product['price']}</p><p><strong>Manufacturer:</strong> <span>{product['manufacturer']}</span></p><div class="detail-buttons-container"><button class="btn" onclick="google.colab.kernel.invokeFunction('add_item_to_cart', ['{product['id']}'], {{}});">Add to Cart</button><button class="btn buy-now-btn" onclick="google.colab.kernel.invokeFunction('handle_direct_buy_now', ['{product['id']}'], {{}});">Buy Now</button></div><button class="btn back-btn" onclick="google.colab.kernel.invokeFunction('show_shop_view', [], {{}});">← Back to Products</button></section>"""

def _calculate_cart_total_and_items():
    total_price = 0.0; cart_display_items = []; item_counts = {}
    for product_id in session['cart']: item_counts[product_id] = item_counts.get(product_id, 0) + 1
    for product_id, quantity in item_counts.items():
        product_details = next((p for p in PRODUCTS if p['id'] == product_id), None)
        if product_details:
            cart_display_items.append({**product_details, 'quantity': quantity})
            try: total_price += float(product_details['price'].replace('₹', '')) * quantity
            except ValueError: pass
    return cart_display_items, total_price

def get_cart_html():
    cart_display_items, total_price = _calculate_cart_total_and_items()
    cart_list_html = "".join([f"""<li><span>{item['name']} (x{item['quantity']}) - {item['price']} each</span><button class="remove-btn btn" onclick="google.colab.kernel.invokeFunction('remove_item_from_cart', ['{item['id']}'], {{}});">Remove One</button></li>""" for item in cart_display_items]) if cart_display_items else "<p style='text-align: center; margin-top: 1rem; color: #666;'>Your cart is empty.</p>"
    total_price_formatted = f"₹{total_price:.2f}"
    checkout_button_html = f'<button class="btn checkout-btn" style="width:auto; padding: 0.85rem 2.5rem;" onclick="google.colab.kernel.invokeFunction(\'show_payment_view\', [{{ \'is_direct_buy\': false }}], {{}});">Proceed to Checkout</button>' if cart_display_items else ""
    return f"""<section class="detail-view"><h2>Your Cart</h2><ul class="cart-items">{cart_list_html}</ul><div class="cart-summary"><p>Total: {total_price_formatted}</p></div>{checkout_button_html}<button class="btn back-btn" style="width:auto; padding: 0.85rem 2.5rem;" onclick="google.colab.kernel.invokeFunction('show_shop_view', [], {{}});">Back to Shop</button></section>"""

def get_payment_form_html(payment_info, error_message=""):
    total_amount_formatted = payment_info['total_amount_formatted']; item_summary_html = payment_info.get('item_summary_html', '')
    back_button_onclick = f"google.colab.kernel.invokeFunction('show_product_detail', ['{payment_info['product_id']}'], {{}});" if payment_info.get('is_direct_buy') and payment_info.get('product_id') else "google.colab.kernel.invokeFunction('show_cart', [], {});"
    back_button_text = "Back to Product" if payment_info.get('is_direct_buy') and payment_info.get('product_id') else "Back to Cart"
    return f"""<div class="form-container" id="paymentForm"><h2>Payment Details</h2>{f'<p class="error-message">{error_message}</p>' if error_message else ""}{item_summary_html}<p class="payment-total">Total Amount: {total_amount_formatted}</p><p style="font-size:0.8rem; text-align:center; margin-bottom:1rem; color: #555;">This is a demo. Please use dummy card details.</p><input type="text" id="card_name" placeholder="Name on Card" value="Demo User"><input type="text" id="card_number" placeholder="Card Number (e.g., 4242-4242-4242-4242)" value="4242424242424242"><input type="text" id="card_expiry" placeholder="Expiry Date (MM/YY)" value="12/29"><input type="text" id="card_cvv" placeholder="CVV (e.g., 123)" value="123"><button onclick="google.colab.kernel.invokeFunction('handle_payment_submit', [document.getElementById('card_name').value, document.getElementById('card_number').value, document.getElementById('card_expiry').value, document.getElementById('card_cvv').value], {{}});">Pay Now {total_amount_formatted}</button><button class="secondary-action-btn" onclick="{back_button_onclick}">{back_button_text}</button></div>"""

def get_order_confirmation_html():
    return """<div class="detail-view confirmation-view"><h2>Payment Successful!</h2><p>Your order has been placed (not really!).</p><p>Thank you for shopping with GlowUp Cosmetics!</p><button class="btn" style="width:auto; padding: 0.85rem 2.5rem;" onclick="google.colab.kernel.invokeFunction('show_shop_view', [], {});">Continue Shopping</button></div>"""

def get_orders_page_html():
    if not MOCK_ORDERS: # In a real app, you'd filter by user ID
        orders_html = "<p class='no-orders'>You have no past orders.</p>"
    else:
        orders_html = ""
        for order in MOCK_ORDERS:
            items_html = "<ul>"
            for item in order['items']:
                items_html += f"<li><span>{item['name']} (x{item['quantity']})</span> <span>{item['price_each']}</span></li>"
            items_html += "</ul>"
            status_class = order['status'].lower().replace(" ", "-") # e.g. 'delivered', 'shipped', 'processing'
            orders_html += f"""
            <div class="order-card">
                <h3>Order ID: {order['order_id']}</h3>
                <p>Date: {order['date']}</p>
                <p>Status: <span class="order-status {status_class}">{order['status']}</span></p>
                <h4>Items:</h4>
                {items_html}
                <p class="order-total">Total: {order['total_amount']}</p>
            </div>
            """
    return f"""<div class="orders-container"><h2>Your Past Orders</h2>{orders_html}<div style="text-align:center; margin-top:1rem;"><button class="btn" onclick="google.colab.kernel.invokeFunction('show_shop_view', [], {{}});">Back to Shop</button></div></div>"""


# --- Backend Functions (Colab Callbacks) ---
def _clear_session_for_new_view():
    """Clears volatile session keys when changing major views, except user auth."""
    session['selected_product'] = None
    session['direct_buy_item_details'] = None
    # Don't clear cart unless specific action like logout or successful payment.

def show_login_view(error_message="", success_message=""):
    clear_output(wait=True); session.update({'current_view': 'login', 'logged_in': False, 'current_user_email': None, 'cart': [], 'direct_buy_item_details': None, 'selected_product': None})
    display(HTML(get_html_template("", 'login', error_message=error_message, success_message=success_message)))

def show_signup_view(error_message="", success_message=""):
    clear_output(wait=True); session.update({'current_view': 'signup', 'logged_in': False, 'current_user_email': None, 'cart': [], 'direct_buy_item_details': None, 'selected_product': None})
    display(HTML(get_html_template("", 'signup', error_message=error_message, success_message=success_message)))

def logout_user():
    # More selective reset to keep the session dict object but clear user data
    session['logged_in'] = False
    session['current_user_email'] = None
    session['cart'] = []
    session['selected_product'] = None
    session['direct_buy_item_details'] = None
    session['current_view'] = 'login'
    clear_output(wait=True)
    show_login_view(success_message="You have been successfully logged out.")


def login_submit(email, password):
    clear_output(wait=True)
    user_account = next((u for u in USERS if u['email'] == email and u['password'] == password), None)
    if user_account:
        session.update({'logged_in': True, 'current_user_email': email, 'current_view': 'shop'})
        _clear_session_for_new_view()
        show_shop_view()
    else: show_login_view(error_message="Invalid email or password.")

def signup_submit(email, password, confirm_password):
    clear_output(wait=True)
    if not all([email, password, confirm_password]): return show_signup_view(error_message="All fields are required.")
    if not ('@' in email and '.' in email): return show_signup_view(error_message="Invalid email format.")
    if password != confirm_password: return show_signup_view(error_message="Passwords do not match.")
    if any(u['email'] == email for u in USERS): return show_signup_view(error_message="Email already registered.")
    USERS.append({'email': email, 'password': password})
    show_login_view(success_message="Signup successful! Please login.")

def show_shop_view(filtered_products=None, search_query=""):
    clear_output(wait=True); session['current_view'] = 'shop'; _clear_session_for_new_view()
    display(HTML(get_html_template(get_shop_html(filtered_products if filtered_products is not None else PRODUCTS, search_query), 'shop')))

def show_product_detail(product_id):
    clear_output(wait=True); session['current_view'] = 'detail'; _clear_session_for_new_view()
    session['selected_product'] = next((p for p in PRODUCTS if p['id'] == product_id), None)
    display(HTML(get_html_template(get_detail_html(session['selected_product']), 'detail')))

def add_item_to_cart(product_id):
    session['cart'].append(product_id)
    clear_output(wait=True)
    alert_product_name = "Item"; current_product_in_view = session.get('selected_product')
    if current_product_in_view and current_product_in_view['id'] == product_id:
        alert_product_name = current_product_in_view['name']
        display(HTML(get_html_template(get_detail_html(current_product_in_view), 'detail')))
    else: # This case should be rare if add to cart is primarily from detail view
        product = next((p for p in PRODUCTS if p['id'] == product_id), None)
        if product: alert_product_name = product['name']
        show_shop_view() # Fallback to shop view
    item_count_in_cart = session['cart'].count(product_id); total_items = len(session['cart'])
    display(HTML(f"<script>alert('✅ {alert_product_name} (Qty: {item_count_in_cart}) added to cart! Total items: {total_items}.');</script>"))

def handle_direct_buy_now(product_id):
    product_to_buy = next((p for p in PRODUCTS if p['id'] == product_id), None)
    if product_to_buy:
        session['direct_buy_item_details'] = product_to_buy
        show_payment_view({'is_direct_buy': True})
    else:
        show_product_detail(product_id) # Should not happen
        display(HTML("<script>alert('Error: Product not found for Buy Now.');</script>"))

def remove_item_from_cart(product_id):
    if product_id in session['cart']: session['cart'].remove(product_id)
    show_cart() # Refresh cart view

def show_cart():
    clear_output(wait=True); session['current_view'] = 'cart'; _clear_session_for_new_view()
    display(HTML(get_html_template(get_cart_html(), 'cart')))

def filter_products(query):
    # No clear_output here, as show_shop_view will handle it.
    filtered = [p for p in PRODUCTS if query.lower() in p['name'].lower() or query.lower() in p['desc'].lower()] if query else PRODUCTS
    show_shop_view(filtered, query) # show_shop_view clears output

def show_payment_view(options=None, error_message=""):
    clear_output(wait=True); session['current_view'] = 'payment'
    is_direct_buy = options.get('is_direct_buy', False) if options else bool(session.get('direct_buy_item_details')) # Persist if coming from error
    payment_info = {'is_direct_buy': is_direct_buy, 'item_summary_html': ''}

    if is_direct_buy and session.get('direct_buy_item_details'):
        item = session['direct_buy_item_details']
        try:
            price_numeric = float(item['price'].replace('₹', ''))
            payment_info['total_amount_formatted'] = f"₹{price_numeric:.2f}"
            payment_info['product_id'] = item['id']
            payment_info['item_summary_html'] = f"<p class='payment-item-summary'>Purchasing: {item['name']} - {item['price']}</p>"
        except ValueError:
            show_product_detail(item['id']); display(HTML("<script>alert('Error: Price calculation failed.');</script>")); return
    else: # Cart checkout
        if not session['cart']:
            show_cart(); display(HTML("<script>alert('Your cart is empty.');</script>")); return
        _, total_price = _calculate_cart_total_and_items()
        payment_info['total_amount_formatted'] = f"₹{total_price:.2f}"
    display(HTML(get_html_template(get_payment_form_html(payment_info, error_message), 'payment')))

def handle_payment_submit(card_name, card_number, card_expiry, card_cvv):
    clear_output(wait=True); errors = []
    if not card_name.strip(): errors.append("Name on card is required.")
    if not re.match(r"^(?:\d[ -]*?){13,19}$", card_number): errors.append("Invalid card number (13-19 digits).")
    if not re.match(r"^(0[1-9]|1[0-2])\/?([0-9]{2})$", card_expiry): errors.append("Invalid expiry (MM/YY).")
    if not re.match(r"^\d{3,4}$", card_cvv): errors.append("Invalid CVV (3-4 digits).")

    is_direct_buy_flow = bool(session.get('direct_buy_item_details'))
    if errors:
        show_payment_view(options={'is_direct_buy': is_direct_buy_flow}, error_message="<br>".join(errors))
    else:
        if is_direct_buy_flow: session['direct_buy_item_details'] = None
        else: session['cart'] = []
        show_order_confirmation_view()

def show_order_confirmation_view():
    clear_output(wait=True); session['current_view'] = 'confirmation'; _clear_session_for_new_view()
    display(HTML(get_html_template(get_order_confirmation_html(), 'confirmation')))

def show_orders_view():
    clear_output(wait=True); session['current_view'] = 'orders'; _clear_session_for_new_view()
    display(HTML(get_html_template(get_orders_page_html(), 'orders')))


# --- Register Callbacks ---
output.register_callback('show_login_view', show_login_view)
output.register_callback('show_signup_view', show_signup_view)
output.register_callback('logout_user', logout_user)
output.register_callback('login_submit', login_submit)
output.register_callback('signup_submit', signup_submit)
output.register_callback('show_shop_view', show_shop_view)
output.register_callback('show_product_detail', show_product_detail)
output.register_callback('add_item_to_cart', add_item_to_cart)
output.register_callback('handle_direct_buy_now', handle_direct_buy_now)
output.register_callback('remove_item_from_cart', remove_item_from_cart)
output.register_callback('show_cart', show_cart)
output.register_callback('filter_products', filter_products)
output.register_callback('show_payment_view', show_payment_view)
output.register_callback('handle_payment_submit', handle_payment_submit)
output.register_callback('show_order_confirmation_view', show_order_confirmation_view)
output.register_callback('show_orders_view', show_orders_view)

# --- Start the application ---
# This initial startup logic helps direct to the correct view based on session state.
# It's simplified; more complex apps might have more robust routing.
if 'current_view' not in session: session['current_view'] = 'login' # Default to login

if not session.get('logged_in', False):
    if session['current_view'] == 'signup': show_signup_view()
    else: show_login_view()
else: # User is logged in
    current_view = session['current_view']
    if current_view == 'login' or current_view == 'signup': # Should not be here if logged in
        show_shop_view()
    elif current_view == 'shop': show_shop_view()
    elif current_view == 'detail' and session.get('selected_product'): show_product_detail(session['selected_product']['id']) # Restore detail view
    elif current_view == 'cart': show_cart()
    elif current_view == 'payment': show_payment_view({'is_direct_buy': bool(session.get('direct_buy_item_details'))}) # Restore payment view
    elif current_view == 'confirmation': show_order_confirmation_view()
    elif current_view == 'orders': show_orders_view()
    else: show_shop_view() # Fallback for any other unexpected state